<a href="https://colab.research.google.com/github/mcneela/Protein-LM-HF-Finetune/blob/master/Protein_LM_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FineTuning DistilProtBert

## Installation and Data Prep

In [ ]:
!pip -q install transformers datasets

In [2]:
# upload GB1 dataset (select from local disk)
from google.colab import files
uploaded = files.upload()

Saving gb1.tsv to gb1.tsv


In [22]:
!wc -l /content/GB1/gb1.tsv
!head -482476 /content/GB1/gb1.tsv >> /content/GB1/train/train.tsv
!tail -53609 /content/GB1/gb1.tsv >> /content/GB1/test/test.tsv

536085 /content/GB1/gb1.tsv


In [51]:
import datasets
data = datasets.load_dataset("GB1")
print(data['train'][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset gb1 downloaded and prepared to /root/.cache/huggingface/datasets/gb1/full_dataset/1.0.0/ebd790e5cda8d93bf33be0a85856438f1336ca2a21f0e9197475baf5a14da116. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'variant': 'Q1A', 'seq': 'MAYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE', 'num_mutations': 1, 'labels': 0.41732627153396606}


## Model Training

In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("yarongef/DistilProtBert")
model = AutoModelForSequenceClassification.from_pretrained("yarongef/DistilProtBert", num_labels=1)

Some weights of the model checkpoint at yarongef/DistilProtBert were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yarongef/DistilProtBe

In [54]:
def tokenize_function(example):
    return tokenizer(example["seq"], truncation=True)


tokenized_datasets = data.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/483 [00:00<?, ?ba/s]

  0%|          | 0/54 [00:00<?, ?ba/s]

In [55]:
from transformers import Trainer, TrainingArguments

# Specifiy the arguments for the trainer  
training_args = TrainingArguments(
    output_dir ='./results',          
    num_train_epochs = 2,     
    per_device_train_batch_size = 64,   
    per_device_eval_batch_size = 20,   
    weight_decay = 0.01,               
    learning_rate = 2e-5,
    logging_dir = './logs',            
    save_total_limit = 10,
    load_best_model_at_end = True,     
    metric_for_best_model = 'rmse',    
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
) 

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq, variant, num_mutations. If seq, variant, num_mutations are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 482475
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15078
  Number of trainable parameters = 230988801


Epoch,Training Loss,Validation Loss
